In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA, KernelPCA
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, ElasticNet, RANSACRegressor, HuberRegressor
from sklearn.svm import SVR

from utils.paths import DATA_RAW_DIR

In [3]:
path_happiness_corrupt = str(DATA_RAW_DIR / "happiness_corrupt.csv")
print(Path(path_happiness_corrupt).exists())

True


In [4]:
df_happy_corrupt = pd.read_csv(path_happiness_corrupt, sep=",", encoding="utf-8")
df_happy_corrupt.head()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [5]:
df_happy_corrupt.describe()

,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
count,167.00000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000
mean,84.00000,4.969299,5.060542,4.878057,0.913960,1.103468,0.511723,0.379412,0.229143,0.114273,1.717287
std,48.35287,1.763655,1.776309,1.752039,0.478880,0.413995,0.269325,0.179122,0.144714,0.102980,0.679444
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,42.50000,4.283000,4.364666,4.184506,0.562392,0.960919,0.302813,0.265565,0.129197,0.045899,1.515013
50%,84.00000,5.235000,5.325246,5.131118,1.000820,1.227619,0.589465,0.423026,0.224151,0.085242,1.789646
75%,125.50000,6.077500,6.179484,5.954942,1.291502,1.402300,0.710039,0.505683,0.315959,0.146237,2.110994
max,167.00000,7.537000,7.622030,7.479556,1.870766,1.610574,0.949492,0.658249,0.838075,0.464308,3.117485


In [6]:
X = df_happy_corrupt.drop(columns=['country', 'score'])
y = df_happy_corrupt['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
estimators = {
    # 'LinearRegression': LinearRegression(),
    # 'Lasso': Lasso(alpha=1.0),
    # 'Ridge': Ridge(alpha=0.5),
    # 'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5),
    'RANSAC': RANSACRegressor(),
    'Huber': HuberRegressor(epsilon=1.35),
    'SVR': SVR(gamma='auto', C=1.0, epsilon=0.1)
}

In [12]:
for name, estimator in estimators.items():
    print(f"Training {name}...")
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2_score = estimator.score(X_test, y_test)

    print(f"Results for {name}:")
    print(f"Mean Squared Error: {mse:.2e}")
    print(f"R^2 Score: {r2_score:.2f}")
    print("\n")


Training RANSAC...
Results for RANSAC:
Mean Squared Error: 1.28e-19
R^2 Score: 1.00


Training Huber...
Results for Huber:
Mean Squared Error: 5.20e-06
R^2 Score: 1.00


Training SVR...
Results for SVR:
Mean Squared Error: 7.46e-02
R^2 Score: 0.97




/opt/anaconda3/envs/ml-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
